## Ejercicio 3 - Grid Search

In [78]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from nltk import word_tokenize
from nltk.corpus import stopwords

from sentiment.tass import InterTASSReader
from sentiment.baselines import MostFrequent
from sentiment.classifier import SentimentClassifier

import pandas as pd

In [22]:
# Loading Training Set
corpus_train = "./InterTASS/ES/intertass-ES-train-tagged.xml"
reader_train = InterTASSReader(corpus_train)
X_train, y_train = list(reader_train.X()), list(reader_train.y())

In [23]:
# Loading Dev Set
corpus_dev = "./InterTASS/ES/intertass-ES-development-tagged.xml"
reader_dev = InterTASSReader(corpus_dev)
X_dev, y_dev = list(reader_dev.X()), list(reader_dev.y())

In [60]:
def eval(model, X, y_true):
    y_pred = model.predict(X)
    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred, average='macro')
    return {'acc': acc, 'f1': f1}

#### Logistic Regression

In [95]:
clf = LogisticRegression()

pipeline_lr = Pipeline([
            ('vect', CountVectorizer(tokenizer=word_tokenize)),
            ('clf', clf),
        ])

In [97]:
param_grid = {
    'clf__penalty': ('l1','l2'),
    'clf__C': [0.001, 0.01, 0.1, 1, 10],
}

params_list = list(ParameterGrid(param_grid))

In [98]:
results = []
for params in params_list:
    pipeline_lr.set_params(**params)
    pipeline_lr.fit(X_train, y_train)
    result = eval(pipeline_lr, X_dev, y_dev)

    results.append({
        **result,
        **params,
    })


/home/camporeale/miniconda3/envs/pln2019/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/camporeale/miniconda3/envs/pln2019/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/camporeale/miniconda3/envs/pln2019/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [99]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)

,acc,clf__C,clf__penalty,f1
5,0.559289,0.100,l2,0.350232
7,0.557312,1.000,l2,0.391317
9,0.537549,10.000,l2,0.384407
6,0.537549,1.000,l1,0.377131
3,0.529644,0.010,l2,0.287032
4,0.527668,0.100,l1,0.293593
8,0.517787,10.000,l1,0.374473
1,0.476285,0.001,l2,0.223608
2,0.436759,0.010,l1,0.157781
0,0.432806,0.001,l1,0.151034


#### SVM

In [100]:
clf = LinearSVC()

pipeline_svm = Pipeline([
            ('vect', CountVectorizer(tokenizer=word_tokenize)),
            ('clf', clf),
        ])

In [101]:
param_grid = {
    'clf__penalty': ['l1','l2'],
    'clf__C': [0.001, 0.01, 0.1, 1, 10],
    'clf__dual': [False] 
}

params_list = list(ParameterGrid(param_grid))

In [102]:
results = []
for params in params_list:
    pipeline_svm.set_params(**params)
    pipeline_svm.fit(X_train, y_train)
    result = eval(pipeline_svm, X_dev, y_dev)

    results.append({
        **result,
        **params,
    })

/home/camporeale/miniconda3/envs/pln2019/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [103]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)

,acc,clf__C,clf__dual,clf__penalty,f1
3,0.567194,0.010,False,l2,0.348590
5,0.561265,0.100,False,l2,0.409613
4,0.547431,0.100,False,l1,0.330368
7,0.531621,1.000,False,l2,0.400324
1,0.527668,0.001,False,l2,0.284634
9,0.515810,10.000,False,l2,0.397132
6,0.511858,1.000,False,l1,0.374765
2,0.511858,0.010,False,l1,0.266006
8,0.505929,10.000,False,l1,0.378207
0,0.432806,0.001,False,l1,0.151034


#### MultinomialNB

In [91]:
clf = MultinomialNB()

pipeline_nb = Pipeline([
            ('vect', CountVectorizer(tokenizer=word_tokenize)),
            ('clf', clf),
        ])

In [92]:
param_grid = {
    'clf__alpha': [1, 0.1, 0.01, 0.0001] 
}

params_list = list(ParameterGrid(param_grid))

In [93]:
results = []
for params in params_list:
    pipeline_nb.set_params(**params)
    pipeline_nb.fit(X_train, y_train)
    result = eval(pipeline_nb, X_dev, y_dev)

    results.append({
        **result,
        **params,
    })

/home/camporeale/miniconda3/envs/pln2019/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [94]:
results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)

,acc,clf__alpha,f1
0,0.565217,1.0000,0.316191
1,0.549407,0.1000,0.399606
2,0.523715,0.0100,0.393394
3,0.507905,0.0001,0.374724


## Ejercicio 4 - Inspección de Modelos

In [104]:
pipeline_lr.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function word_tokenize at 0x7fd71663ef28>,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]

In [105]:
vect = pipeline_lr.named_steps['vect']
clf = pipeline_lr.named_steps['clf']

In [106]:
features = vect.get_feature_names()
len(features)
features[:10]

['!', '#', '$', '%', "'", "''", "'chicas", "'iguales", '(', ')']

In [107]:
coef = clf.coef_
coef.shape

(4, 5049)

In [108]:
import pandas as pd
features_df = pd.DataFrame({'name': features, 'coef': coef.ravel()})
features_df.sort_values('coef', inplace=True)

ValueError: arrays must all be same length